In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pickle
import argparse

# parser = argparse.ArgumentParser(description='This is a demonstration program')
# # parser.add_argument('-batch_size', type=str, required=True, help='Please provide a batchsize')
# args = parser.parse_args()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

batch_size = 64
block_size = 128
max_iters = 10000
lr = 1e-4
eval_iters = 500
eval_interval = 500
n_embed = 384 # number of elements in the embedding vector
n_head = 16
n_layer = 16 #num of decoder layers
dropout = 0.2 # Dropout probability (makes model generalize more)

In [2]:
device

'cuda'

In [3]:
chars = ""
with open('vocab.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(set(text))


vocab_size = len(chars)
vocab_size


32171

In [4]:
# This creates a dictionary mapping strings to int and int to string
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
# Turns text into int
encode = lambda s: [string_to_int[c] for c in s]
# Turns into into text
decode = lambda l: ''.join([int_to_string[i] for i in l])


In [5]:
import mmap
import random

In [6]:
def get_random_chunk(split):
    filename = "train_split.txt" if split == 'train' else 'val_split.txt'
    # Open file in binary
    with open(filename, 'rb') as f:
        # Open with memory mapping
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            file_size = len(mm)
            # Making end size before actual end so no memory out of bounds errors
            start_pos = random.randint(0,(file_size) - block_size*batch_size)
    
            mm.seek(start_pos)
            block = mm.read(block_size * batch_size - 1)
            # Decode
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
    return data




def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y



In [7]:
@torch.no_grad()
def estimate_loss(model:nn.Module):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model.forward(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return  out

Consider a batch of text sequences represented as a tensor with shape \( B \times T \times C \):

- **Batch Size (B)**: If \( $B = 64$ \), there are 64 sequences in the batch.
- **Sequence Length (T)**: If \( $T = 128$ \), each sequence has 128 tokens.
- **Channel Dimension (C)**: If \( $C = 512$ \), each token is represented by a 512-dimensional embedding vector.

In [8]:
class Head(nn.Module):
    '''one head of self-attention'''
    def __init__(self, head_size):
        super().__init__()
        # Wk, Wq, Wv parameters
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -.5 # That end bit is just sqrt d_k
        wei = wei.masked_fill(self.tril[:T, :T] ==0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embed)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # For every head, forward on each of them and cat them
        out = self.dropout(self.proj(out))
        return out
class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(    nn.Linear(n_embed, 4 * n_embed), nn.ReLU(), nn.Linear(4*n_embed, n_embed), nn.Dropout(dropout))
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    # Decoder Block
    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head #num of features each head captures
        self.sa = MultiHeadAttention(n_head, head_size) # Self Attention
        self.ffwd = FeedForward(n_embed) # Feed Forward Linear -> ReLU -> Linear
        self.ln1 = nn.LayerNorm(n_embed) # LayerNorm
        self.ln2 = nn.LayerNorm(n_embed) # LayerNorm
    def forward(self,x):
        y = self.sa(x) # Y is self attention and x is added later
        x = self.ln1(x+y) # Residual connection for self attention layer
        
        y = self.ffwd(x)
        x = self.ln2(x+y) # resiudual connection for feed forwards
        return x


In [9]:
temp = torch.rand(128, 64, 384)
tempM = Block(n_embed, n_head)
tempM(temp)

tensor([[[-0.3745, -0.6510, -0.7120,  ..., -0.5765,  1.7241,  2.0743],
         [ 1.2201, -0.7214, -1.0183,  ..., -1.8096,  0.7192,  1.1727],
         [ 1.1013, -1.2056, -0.1245,  ..., -0.9044, -0.5779,  0.9289],
         ...,
         [ 0.8428,  0.2611, -0.5450,  ..., -0.3930, -0.8016, -0.0912],
         [ 1.8818,  0.6479,  0.2682,  ..., -2.1176,  0.6618,  0.2705],
         [ 0.8391,  0.4207, -1.9776,  ...,  0.6284, -1.1361, -0.2795]],

        [[ 1.2216, -1.8785,  0.1243,  ..., -0.0300, -0.9993,  0.5460],
         [ 1.4146, -0.1245, -1.1276,  ...,  0.1211, -0.6716, -1.5616],
         [ 0.5980, -1.5023,  0.5972,  ...,  0.5012, -1.1951,  1.2981],
         ...,
         [-0.3415, -0.3822, -1.4461,  ..., -2.3127, -0.0408,  0.3911],
         [ 0.3505,  0.8841, -0.1659,  ..., -0.9632,  1.1264, -0.7043],
         [ 0.5089,  0.8550, -1.1818,  ..., -0.5661,  1.7566, -0.2704]],

        [[ 0.3528, -1.4555, -0.8705,  ..., -2.5673,  0.1229,  1.0867],
         [-0.1983,  1.0654, -1.4940,  ..., -1

In [10]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # This is the token embedding 
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table=nn.Embedding(block_size, n_embed)
        # decoderblocks (temp code for the future) 
        # We're using embeddings nn because sinosoidual is mainly for base transformer models
        # We're building GPT, and that's what experts do. Transformers vs. GPT
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        self.ln_F = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
    def _init_weights(self, module):
        torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        if module.bias is not None:
            torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    def forward(self, index, targets=None):

        B, T = index.shape
        # 
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        # print(f'Pre')
        x = tok_emb + pos_emb
        # print(f'tok + pos {x.shape}')
        x = self.blocks(x)
        # print(f'X selfblocks shape {x.shape}')
        x = self.ln_F(x) # Layer Norm
        logits = self.lm_head(x) # Linear Transformation

        if targets is None:
            loss = None 
        else:
            # B -> Batch
            # T-> Seq Len
            #C -> feature dimension
            B,T,C = logits.shape
            # Reshaping for the loss function
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # The logits and targets are flattened for the loss function which only
            # Takes of these dimensions
            loss = F.cross_entropy(logits,targets)
        return logits, loss
    def generate(self, index, max_new_tokens):
        #index is (B,T) array of indicies in the current context
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step (selects logits for previous batch)
            logits = logits[:, -1, :] 
            # apply softmax for probs
            probs=F.softmax(logits, dim=-1) # (B,C)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = GPTLanguageModel(vocab_size)

m = model.to(device)
# context = torch.zeros((1,1), dtype=torch.long, device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500[0].toList()))
#print(generated_chars)   


In [13]:
# Create a PyTorch optimizer
from tqdm import tqdm
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

for iter in tqdm(range(max_iters), desc="Training Progress"):
    if iter % 100 ==0:
        print(iter)
    if iter % eval_iters == 0:
        losses = estimate_loss(model)
        print(f'step: {iter}, train loss: {losses["train"]:.3f}, val loss: {losses["val"]:.3f}')  # Fix quotes around dictionary keys
    # Sample a batch of data
    xb, yb = get_batch('train')
    # Evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


Training Progress:   0%|          | 0/10000 [00:00<?, ?it/s]

0
step: 0, train loss: 1.500, val loss: 1.490


Training Progress:   1%|          | 100/10000 [05:15<4:33:40,  1.66s/it] 

100


Training Progress:   2%|▏         | 200/10000 [08:01<4:28:23,  1.64s/it]

...

5200


Training Progress:  53%|█████▎    | 5300/10000 [3:00:00<2:09:00,  1.65s/it]

5300


Training Progress:  54%|█████▍    | 5399/10000 [3:02:45<2:35:44,  2.03s/it]


KeyboardInterrupt: 

In [41]:
import pickle

In [14]:
with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

model saved


In [15]:
print("loading model params")
with open('model-01.pkl', 'rb') as f:
    model = pickle.load(f) 
print('loaded successfully')


loading model params
loaded successfully


In [16]:
# Calc Loss
estimate_loss(model)

{'train': tensor(1.4215), 'val': tensor(1.4314)}

In [17]:
prompt = input("Prompt:\n")
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context.unsqueeze(0), max_new_tokens=150)[0].tolist())
print(f'Completion:\n {generated_chars}')

Completion:
 This is how you make money, first step: I'm a head at a Company _and the upkerget arms who among there I had, behind and diving her with announces by the John Jiana and the".[Lew Abiaclo da


### Torch Load & Save